In [39]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import csr_matrix

In [ ]:
# Пример данных о фильмах
movies = {
    'movie_id': [1, 2, 3, 4],
    'title': ['Inception', 'Interstellar', 'The Dark Knight', 'Memento'],
    'description': [
        'A mind-bending thriller set within the architecture of the mind.',
        'A team of explorers travel through a wormhole in space in an attempt to ensure humanity\'s survival.',
        'When the menace known as The Joker emerges, Batman must step up to save Gotham.',
        'A man with short-term memory loss attempts to track down his wife\'s murderer.'
    ]
}

df_movies = pd.DataFrame(movies)

In [4]:
book = pd.read_csv('book\\Books.csv')
ratings = pd.read_csv('book\\Ratings.csv')
book

C:\Users\Gleb\AppData\Local\Temp\ipykernel_18428\3069710769.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('book\\Books.csv')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [13]:
book.info()

<class 'pandas.core.frame.DataFrame'>
Index: 271353 entries, 0 to 271359
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271353 non-null  object
 1   Book-Title           271353 non-null  object
 2   Book-Author          271353 non-null  object
 3   Year-Of-Publication  271353 non-null  object
 4   Publisher            271353 non-null  object
 5   Image-URL-S          271353 non-null  object
 6   Image-URL-M          271353 non-null  object
 7   Image-URL-L          271353 non-null  object
 8   description          271353 non-null  object
dtypes: object(9)
memory usage: 20.7+ MB


In [12]:
book.dropna(inplace=True)

In [14]:
book['description'] = book['Book-Title'] + ' ' + book['Book-Author'] + ' ' + book['Publisher']
book

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,description
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Classical Mythology Mark P. O. Morford Oxford ...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,Clara Callan Richard Bruce Wright HarperFlamin...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,Decision in Normandy Carlo D'Este HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,The Mummies of Urumchi E. J. W. Barber W. W. N...
...,...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,There's a Bat in Bunk Five Paula Danziger Rand...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,From One to One Hundred Teri Sloat Dutton Books
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,Lily Dale : The True Story of the Town that Ta...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,Republic (World's Classics) Plato Oxford Unive...


In [51]:
new_book = book.head(1000)

In [53]:
# Используем TF-IDF для векторизации описаний фильмов
tfidf_vectorizer = HashingVectorizer(stop_words='english', n_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(new_book['description'])

# Убедитесь, что tfidf_matrix является разреженной матрицей
tfidf_matrix_csr = csr_matrix(tfidf_matrix)

# Рассчитываем косинусную близость
cosine_sim = cosine_similarity(tfidf_matrix_csr)

In [58]:
# Функция для получения похожих фильмов
def get_recommendations(title, cosine_sim=cosine_sim):
    # Находим индекс фильма по названию
    idx = book[book['Book-Title'] == title].index[0]

    # Находим косинусную похожесть данного фильма с остальными
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Сортируем фильмы по степени похожести
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Получаем индексы самых похожих фильмов
    sim_scores = sim_scores[1:4]  # Пропускаем сам фильм
    movie_indices = [i[0] for i in sim_scores]

    # Возвращаем названия похожих фильмов
    return book['Book-Title'].iloc[movie_indices]

# Получение рекомендаций
title = 'Classical Mythology'
recommended_movies = get_recommendations(title)
print(f'Recommended movies for "{title}"')
recommended_movies

Recommended movies for "Classical Mythology"


817    How Not to Say What You Mean: A Dictionary of ...
397            Julius Caesar (Oxford School Shakespeare)
521                      Cranford (The World's Classics)
Name: Book-Title, dtype: object